# Supplement Figures: S3-S5

3-4 in 18_hadcm3-ppe.ipynb

5 in 31_final-ms-figures.ipynb

In [ ]:
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmocean.cm as cmo

import ppe_plot as ppeplot
import ppe_load as ppeload
import ppe_analyze as ppealyz

%load_ext watermark
%watermark -iv

In [ ]:
# Get plotting settings
ppeplot.setup_plotting_workspace()

## S3: Zonal-mean change in surface albedo and TLAI for HadCM3C Hist PPE

In [ ]:
fig, axes = plt.subplots(
    4, 4,
    figsize=(12, 8),
    sharex=True,
    layout='constrained'
)
alb_ax = axes.ravel()
lai_ax = np.empty_like(alb_ax)

n_lines = 5
cmap = mpl.colormaps['viridis']
colors = cmap(np.linspace(0, 1, n_lines))

for i in had_member:
    lai_ax[i] = alb_ax[i].twinx()

    lai_change_zmlnd = (tavg_had_ppe['total_leaf_area_index'].isel(member=i) - tavg_had_std['total_leaf_area_index']).where(had_weights['landweights']>0).mean(dim='lon')
    lai_change_zmlnd.plot(
        ax=lai_ax[i],
        c='green',
        label='total_lai'
    )

    alb_change_zmlnd = (tavg_had_ppe['surface_albedo'].isel(member=i) - tavg_had_std['surface_albedo']).where(had_weights['landweights']>0).mean(dim='lon')
    alb_change_zmlnd.plot(
        ax=alb_ax[i],
        c='k',
        ls='--',
        label='surface_albedo')

    alb_ax[i].set_title(i['member_name'].values)

for a in alb_ax:
    a.axhline(0, -90, 90, c='gray', zorder=0)
    a.set_ylim(-0.12, 0.12)
    a.set_xlim(-65, 90)
    a.invert_yaxis()
    a.set_xlabel(None)
    a.set_ylabel(None)

for a in lai_ax:
    a.set_ylim(-3, 3)
    a.set_xlim(-65, 90)
    a.set_xlabel(None)
    a.set_ylabel(None)
    a.set_title(None)

for a in alb_ax[-4:]:
    a.set_xlabel('Latitude [$^{\circ}$N]')

for i in range(4):
    alb_ax[i*4].set_ylabel('$\Delta$ Surface albedo')

for i in range(4):
    lai_ax[i*4+3].set_ylabel('$\Delta$ total LAI', color='green')

fig.suptitle('HadCM3 Hist: zonal mean change in surface albedo and total LAI');

## S4: Zonal-mean change in surface albedo and TLAI for HadCM3C A1B PPE

In [ ]:
fig, axes = plt.subplots(
    4, 4,
    figsize=(12, 8),
    sharex=True,
    layout='constrained'
)
alb_ax = axes.ravel()
lai_ax = np.empty_like(alb_ax)

n_lines = 5
cmap = mpl.colormaps['viridis']
colors = cmap(np.linspace(0, 1, n_lines))

for i in had_member:
    lai_ax[i] = alb_ax[i].twinx()

    lai_change_zmlnd = (tavg_hada1b_ppe['total_leaf_area_index'].isel(member=i) - tavg_hada1b_std['total_leaf_area_index']).where(had_weights['landweights']>0).mean(dim='lon')
    lai_change_zmlnd.plot(
        ax=lai_ax[i],
        c='green',
        label='total_lai'
    )

    alb_change_zmlnd = (tavg_hada1b_ppe['surface_albedo'].isel(member=i) - tavg_hada1b_std['surface_albedo']).where(had_weights['landweights']>0).mean(dim='lon')
    alb_change_zmlnd.plot(
        ax=alb_ax[i],
        c='k',
        ls='--',
        label='surface_albedo')

    alb_ax[i].set_title(i['member_name'].values)

for a in alb_ax:
    a.axhline(0, -90, 90, c='gray', zorder=0)
    a.set_ylim(-0.12, 0.12)
    a.set_xlim(-65, 90)
    a.invert_yaxis()
    a.set_xlabel(None)
    a.set_ylabel(None)

for a in lai_ax:
    a.set_ylim(-3, 3)
    a.set_xlim(-65, 90)
    a.set_xlabel(None)
    a.set_ylabel(None)
    a.set_title(None)

for a in alb_ax[-4:]:
    a.set_xlabel('Latitude [$^{\circ}$N]')

for i in range(4):
    alb_ax[i*4].set_ylabel('$\Delta$ Surface albedo')

for i in range(4):
    lai_ax[i*4+3].set_ylabel('$\Delta$ total LAI', color='green')

fig.suptitle('HadCM3 A1B: zonal mean change in surface albedo and total LAI');

## S5: Time series of atmospheric CO2 and corresponding radiative forcing for the HadCM3C PPE

In [ ]:
hadcm_weights = xr.open_dataset('/glade/work/bbuchovecky/HadCM3_analysis/weights/HadCM3_weights_v2.nc')

hadhst_co2massfrac = xr.open_dataset("/glade/work/bbuchovecky/HadCM3_analysis/control/annual_timeseries/HadCM3_PPE.control.ann_mean_ts.mass_fraction_of_carbon_dioxide_in_air.nc")["mass_fraction_of_carbon_dioxide_in_air"]
hada1b_co2massfrac = xr.open_dataset("/glade/work/bbuchovecky/HadCM3_analysis/a1b/annual_timeseries/HadCM3_PPE.a1b.ann_mean_ts.mass_fraction_of_carbon_dioxide_in_air.nc")["mass_fraction_of_carbon_dioxide_in_air"]

# Convert mass fraction to mixing ratio
hadhst_co2ppm = ppealyz.mass_fraction_to_ppm(hadhst_co2massfrac, 44.01).rename("mixing_ratio_of_carbon_dioxide_in_air").weighted(hadcm_weights.globalweights).mean(dim=["lat", "lon"])
hada1b_co2ppm = ppealyz.mass_fraction_to_ppm(hada1b_co2massfrac, 44.01).rename("mixing_ratio_of_carbon_dioxide_in_air").weighted(hadcm_weights.globalweights).mean(dim=["lat", "lon"])

# Deal with data quality issues
hadhst_co2ppm[6, :] = hadhst_co2ppm[6, :].where((hadhst_co2ppm.year != hadhst_co2ppm.year[-1]))
hada1b_co2ppm[8, :] = hada1b_co2ppm[8, :].where((hada1b_co2ppm.year != hada1b_co2ppm.year[-23]))

# Concat into a single time series
hadfull_co2ppm = xr.concat([hadhst_co2ppm, hada1b_co2ppm], dim="year").interpolate_na(dim='year', method='linear')

# Compute the radiative forcing relative to the first 20 years
hadref_co2ppm = hadfull_co2ppm.isel(year=slice(0,20)).mean()
hadfull_co2rf = ppealyz.co2_radiative_forcing(hadfull_co2ppm, hadref_co2ppm)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8,4), layout='constrained')
ax = axes.ravel()

for m in hadfull_co2ppm.member:
    hadfull_co2ppm.sel(member=m).plot(ax=ax[0], c='green', lw=0.8)
    hadfull_co2rf.sel(member=m).plot(ax=ax[1], c='blue', lw=0.8)

hadfull_co2ppm_endmin = hadfull_co2ppm.isel(year=-1).min(dim='member').values
hadfull_co2ppm_endmax = hadfull_co2ppm.isel(year=-1).max(dim='member').values

ax[0].annotate(
    "", [2110, hadfull_co2ppm_endmin], [2110, hadfull_co2ppm_endmax],
    arrowprops=dict(arrowstyle="<|-|>", color="green", lw=2, shrinkA=0, shrinkB=0,
                    mutation_scale=10),
)

ax[0].text(
    2119, (hadfull_co2ppm_endmin+hadfull_co2ppm_endmax)/2,
    f'{abs(hadfull_co2ppm_endmin-hadfull_co2ppm_endmax):0.0f} ppm',
    fontsize=8, color='green',
    rotation=90,
    ha='center', va='center',
)

hadfull_co2rf_endmin = hadfull_co2rf.isel(year=-1).min(dim='member').values
hadfull_co2rf_endmax = hadfull_co2rf.isel(year=-1).max(dim='member').values

ax[1].annotate(
    "", [2110, hadfull_co2rf_endmin], [2110, hadfull_co2rf_endmax],
    arrowprops=dict(arrowstyle="<|-|>", color="blue", lw=2, shrinkA=0, shrinkB=0,
                    mutation_scale=10),
)
ax[1].text(
    2119, (hadfull_co2rf_endmin+hadfull_co2rf_endmax)/2,
    f'{abs(hadfull_co2rf_endmin-hadfull_co2rf_endmax):0.2f} W m$^{{-2}}$',
    fontsize=8, color='blue',
    rotation=90,
    ha='center', va='center',
)

for a in ax:
    a.set_title("")
    a.axvspan(1940, 1990, alpha=0.15, color="gray", linewidth=0)
    a.axvspan(2050, 2100, alpha=0.15, color="gray", linewidth=0)
    a.axvline(1990, color='k', lw=plt.rcParams['axes.linewidth'], ls=':', zorder=0)
    a.text(0.385, 0.99, 'Historical', fontsize=10, rotation=90, fontweight='bold', color='gray', ha='left', va='top', transform=a.transAxes)
    a.text(0.79, 0.99, 'A1B', fontsize=10, rotation=90, fontweight='bold', color='gray', ha='left', va='top', transform=a.transAxes)

ax[0].set_ylim(200, 1200)
ax[0].set_xlim(1853, 2127)
ax[0].set_title(f'(a) Atmospheric CO$_2$ concentration', loc="left")
ax[0].set_ylabel('Atmospheric CO$_2$ [ppm]')
ax[0].set_xlabel('Year')

ax[1].set_ylim(-0.5, 7.5)
ax[1].set_xlim(1853, 2127)
ax[1].set_title(f'(b) Radiative forcing', loc="left")
ax[1].set_ylabel('Radiative forcing [W m$^{-2}$]')
ax[1].set_xlabel('Year')

plt.show()
fig.savefig(f'{export_dir}/supp_hadcm3_co2_rf.pdf', dpi=export_dpi, bbox_inches='tight')

In [ ]:
print(f"{hadfull_co2ppm.sel(year=slice(1940, 1989)).mean(dim="year").mean().values: 0.3f}")
print(f"{hadfull_co2ppm.sel(year=slice(1940, 1989)).mean(dim="year").min().values: 0.3f}")
print(f"{hadfull_co2ppm.sel(year=slice(1940, 1989)).mean(dim="year").max().values: 0.3f}")

In [ ]:
print(f"{hadfull_co2ppm.sel(year=slice(2051, 2100)).mean(dim="year").mean().values: 0.3f}")
print(f"{hadfull_co2ppm.sel(year=slice(2051, 2100)).mean(dim="year").min().values: 0.3f}")
print(f"{hadfull_co2ppm.sel(year=slice(2051, 2100)).mean(dim="year").max().values: 0.3f}")